# Music and iteration

Part of the power of computer programming is telling a computer to do the same instructions over and over, but changing something each time. This is the power of iteration, which typically comes in two flavours

1. `for` loop: iterate over a finite collection of things or until a condition is met
2. `while` loop: do something until a condition is met


For our purposes we are going to mainly look at the `for` loop.

We can demonstrate the utility of `for` in two dimensions of music harmony and rhythm.



## Setup

Lets begin by gathering the materials we will need. Some maths functions and a way to quickly listen to audio

In [ ]:
from math import sin, pi
from IPython.display import Audio


## Iteration and time

Let say we wanted to make a sine wav we will need first our frequency $f_0$, our sampling rate, our duration

In [ ]:
f0 = 440.0     # Fundamental frequency
duration = 1.0 # in seconds
fs = 44100.0   # Sampling Rate 



We also then need to figure out how much we need to change angle of the wave for it to be the correct frequency

In [ ]:
delta = 2.0 * pi * f0 / fs # how much does the phase change between samples

A digital sine wav is made up pf many samples, it would be maddening if we had to write

In [ ]:
sine_wave_sample_0  = sin(delta * 0)
sine_wave_sample_1  = sin(delta * 1)
sine_wave_sample_2  = sin(delta * 2)
sine_wave_sample_3  = sin(delta * 3)
sine_wave_sample_4  = sin(delta * 4)
sine_wave_sample_5  = sin(delta * 5)
sine_wave_sample_6  = sin(delta * 6)
sine_wave_sample_7  = sin(delta * 7)
sine_wave_sample_8  = sin(delta * 8)
sine_wave_sample_9  = sin(delta * 9)
sine_wave_sample_10 = sin(delta * 10)

we'd have to do that another few thousand times before we had eough samples to listen to.

Instead we can use a `for` loop make the list of samples for us.

First we figure out how many samples we need. Which just our duration $\times$ our sample rate.

It can only be an integer as we can't have a fraction of a sample

In [ ]:
num_samples = int(duration*fs)

Next we create a list to which we can add our samples

In [ ]:
sine_wave = []

Then our `for` loop, which states thats an index, `i`, is going step through the numbers in a range of numbers between `0` and `num_samples`. That range is provided by the `range` function

In [ ]:
for i in range(num_samples):
    sine_wave.append([sin(delta * i)])


We can actually write this in a slight more nicer way by making the list directly.

If we wanted a list of numbers from `0` to `10` we could write

In [ ]:
[number for number in range(10)]

The same applies to our sine wave as we could jiust as well write

In [ ]:
sine_wave = [sin(delta * i) for i in range(num_samples)]

Audio(data=sine_wave, rate=fs)

## Iteration and harmonics

now maybe we want to play a fifth interval above that, or $\frac{3}{2}f_0$ for our fundamental frequency $f_0$

In [ ]:
f1 = 3/2 * f0
delta = 2.0 * pi * f1 / fs # how much does the phase change between samples
sine_wave_1 = [sin(delta * i) for i in range(int(duration*fs))]
Audio(data=sine_wave_1, rate=fs)

we needed to write slightly less, but we still need to add the two tones together. We can use iteration for that as well. We can zip them together with the `zip` function

In [ ]:
both_sine_waves = [samp1+samp2 for samp1,samp2 in zip(sine_wave,sine_wave_1)]

Audio(data=both_sine_waves, rate=fs)

But what if we want a third sine wave? or a fourth? twenty? All of sudden this approach doesn't scale very well.

That's where we can add another `for` loop.

In [ ]:
sum_of_sines = []

num_harmonics = 10

for i in range(num_samples):
    sum_of_sines.append(0.0)
    for k in range(num_harmonics):
        delta = 2.0 * pi * f0 * k / fs 
        sum_of_sines[i] += sin(delta * i)
    

That is probably pretty loud, but we can check

In [ ]:
max(sum_of_sines)

That is far too loud, so next we normalise the audio so it is in the range `-1.0` > `+1.0`

In [ ]:
maximum = max(sum_of_sines)

for i in range(num_samples):
    sum_of_sines[i] *= 1.0 / maximum
    
Audio(data=sum_of_sines, rate=fs)

Scaling the amplitude of the harmonic inverse to its frequency should result in something a little more pleasant.

Adding in the line

```py
gain = 1.0 / (f0 * k)
```

In [ ]:
sum_of_sines = []

num_harmonics = 10

for i in range(num_samples):
    sum_of_sines.append(0.0)
    for k in range(num_harmonics):
        gain = 1.0 / (f0 * (k+1))
        delta = 2.0 * pi * f0 * k / fs 
        sum_of_sines[i] += gain*sin(delta * i)
        
maximum = max(sum_of_sines)

for i in range(num_samples):
    sum_of_sines[i] *= 1.0 / maximum
    
Audio(data=sum_of_sines, rate=fs)

The `range` also allows for the starting index and step size to be changed, e.g. `range(0,10,2)` instructs to start on `0`, go up to (but not including) `10` and increase in steps of `2`

By just changing the number and step of the harmonics, we can change the timbre of our sound

For a square wave we would only want the odd numbers

In [ ]:
sum_of_sines = []

num_harmonics = 10

for i in range(num_samples):
    sum_of_sines.append(0.0)
    for k in range(1,num_harmonics,2):
        gain = 1.0 / (f0 * (k+1))
        delta = 2.0 * pi * f0 * k / fs 
        sum_of_sines[i] += gain*sin(delta * i)
        
maximum = max(sum_of_sines)

for i in range(num_samples):
    sum_of_sines[i] *= 1.0 / maximum
    
Audio(data=sum_of_sines, rate=fs)

Or a triangle wave

$$\frac{8}{\pi^2}\sum_{n=0}^{N-1} \frac{{(-1)}^n}{(2n + 1)^2} \sin(2 \pi f_0 (2n + 1) t)$$


where the gain is: $\frac{{(-1)}^n}{(2n + 1)^2}$

and the harmonics are at frequencies: $f_0 (2n + 1)$

In [ ]:
sum_of_sines = []

num_harmonics = 10

for i in range(num_samples):
    sum_of_sines.append(0.0)
    for n in range(0,num_harmonics):
        gain = ((-1.0)**(n)) / ((2*n+1)**2)
        delta = 2.0 * pi * f0 * (2*n+1) / fs 
        sum_of_sines[i] += gain*sin(delta * i)
        
maximum = max(sum_of_sines)

for i in range(num_samples):
    sum_of_sines[i] *= 1.0 / maximum
    
Audio(data=sum_of_sines, rate=fs)

## Iteration in Rhythm

We have seen how we can use a `for` loop can stack frequencies, lets see how we use it for rhythm




Remember our `note_length_samples` is a number of samples. We can't have fractional samples, so it needs to be an `int`

In [ ]:
melody = []

sampling_rate = 44100
beats_per_minute = 120
beats = 8
beat_length = 60 / beats_per_minute
note_length_seconds = beat_length / 2
note_length_samples = int(note_length_seconds * sampling_rate)

f0 = 440
delta = 2.0 * pi * f0 / sampling_rate


note = [sin(delta * i) for i in range(note_length_samples)]
rest = [0.0 for x in range(note_length_samples)]


for i in range(beats):
        melody.extend(note)
        melody.extend(rest)
        
Audio(data=melody, rate=sampling_rate)

In [ ]:
melody = []

sampling_rate = 44100
beats_per_minute = 240
beats = 8
beat_length = 60 / beats_per_minute
note_length_seconds = beat_length / 2
note_length_samples = int(note_length_seconds * sampling_rate)

f0 = 440

rest = [0.0 for x in range(note_length_samples)]


a_major = [440.0,494.0,554.0,622.0,659.0,740.0,831.0,880.0]

notes = []

for i in range(len(a_major)):
    delta = 2.0 * pi * a_major[i] / sampling_rate
    note = [sin(delta * i) for i in range(note_length_samples)]
    notes.append(note)



seems quite messy, maybe rather than using an index, we can iterate over the frequencies directly

In [ ]:
notes = []

for freq in a_major:
    delta = 2.0 * pi * freq / sampling_rate
    note = [sin(delta * i) for i in range(note_length_samples)]
    notes.append(note)

In [ ]:
for i in range(beats):
        melody.extend(notes[i])
        melody.extend(rest)
        
Audio(data=melody, rate=sampling_rate)

## A Simple Sequencer

A sequencer is can be though of as a two dimensional array. One dimension represents pitch and the other represents time.

Pitch and time are in fixed, discrete steps.

We will need a list of lists, a 2D array, where each sub-list represents a beat and which notes should be played in it.

We can quickly generate such a list by using a random number generator

First we import the `random` library

In [ ]:
from random import random

From there we need to create a list of notes. For ease we can use MIDI note numbers

In [ ]:
midi_notes = [96, 93, 91, 89, 86, 84, 81, 79, 77, 74, 72, 69, 67, 65, 62, 60]

Then translate this list into frequencies. Here is a handy function to do so:

In [ ]:
def midi_number_to_freq(midiNoteNumber):
    return 2 ** ((midiNoteNumber - 69.0) / 12.0) * 440.0;


In [ ]:
note_freqs = [midi_number_to_freq(midi_note) for midi_note in midi_notes]

Starting with the first beat, we need a list of `16` values. Each values corresponds to a note and wether it should be played. Lets say a `1` means a note is played and a `0` means it stays silent

The `random()` function will give us a number between `0.0` and `1.0`.


In [ ]:
random()

To turn this into a `1` or a `0` we can say if is smaller, or bigger, than `0.5`

In [ ]:
1 if random() < 0.5 else 0

We can actually play with the "chance" that a not will be played by changing `0.5`, which is a 50% chance. Notes are more interesting if there is more silence around them. Lets begin by having a 15% chance of playing a note

In [ ]:
1 if random() < 0.15 else 0

then all we need is a list of 16, one for each frequency

In [ ]:
[1 if random() < 0.15 else 0 for note in range(16)]

and then we can create a list for every beat. Lets make a square sequencer, with `16` notes per beat and `16` beats in total

In [ ]:
sequence = [[1 if random() < 0.05 else 0 for note in range(16)] for beat in range(16)]
sequence

Notes are read left to right and beats are read a top to bottom.



Like before we can choose our bpm and how long our notes and rests are.

In [ ]:
beats_per_minute = 120
beat_length = 60 / beats_per_minute
note_length_seconds = beat_length / 16
note_length_samples = int(note_length_seconds * sampling_rate)
rest_length_seconds = note_length_seconds*3
note_length_samples = int(rest_length_seconds * sampling_rate)

To save time, we can create a list of notes from which we can choose. We can also make a rest of silence to use between notes

In [ ]:
notes = []

for freq in note_freqs:
    delta = 2.0 * pi * freq / sampling_rate
    note = [sin(delta * i) for i in range(note_length_samples)]
    notes.append(note)

rest = [0.0 for x in range(note_length_samples)]



Now we loop over our sequence. The sequence is made of beats, therefore for each beat in the sequence:

```py
for beat in sequence:
```

we want to stack up notes, which we can call a `chord`

Then for each beat we check if the not should be played. The `enumerate` function will give us both the index and the value

```py
    for note_num, should_play_note in enumerate(beat):
        if should_play_note:
```

but we could also have written

```
    for i in range(len(beat)):
        if beat[i]:     
```

In [ ]:
melody = []

for beat in sequence:
    chord = [0.0 for x in range(note_length_samples)]
    for note_num, should_play_note in enumerate(beat):
        if should_play_note:
            chord = [x + y for x, y in zip(chord, notes[note_num])]
    melody.extend(chord)
    melody.extend(rest)
             
Audio(data=melody, rate=sampling_rate)

### Repetion legitimises

all we need to do is repeat our sequence, and we have a melody line.

Add in a `for` loop

In [ ]:
melody = []
number_of_cycles = 4

for i in range(number_of_cycles):
    for beat in sequence:
        chord = [0.0 for x in range(note_length_samples)]
        for note_num,should_play_note in enumerate(beat):
            if should_play_note:
                chord = [x + y for x, y in zip(chord, notes[note_num])]
        melody.extend(chord)
        melody.extend(rest)
        
Audio(data=melody, rate=sampling_rate)

What if we changed our sequence after 4 cyles?

Add in a `for` loop

In [ ]:
melody = []
number_of_cycles = 4
number_of_sequences = 4

for j in range(number_of_sequences):
    for i in range(number_of_cycles):
        for beat in sequence:
            chord = [0.0 for x in range(note_length_samples)]
            for note_num,should_play_note in enumerate(beat):
                if should_play_note:
                    chord = [x + y for x, y in zip(chord, notes[note_num])]
            melody.extend(chord)
            melody.extend(rest)
            
    sequence = [[1 if random() < 0.15 else 0 for note in range(16)] for beat in range(16)]
        

Audio(data=melody, rate=sampling_rate)

We have a sequences of sequences, but we never repeat it.

Remember, repetition legitimises, therefore we can add another `for` loop

If we repeat the sequences, we can't just create a random one every time, so we will need to remember them.

In [ ]:
sequences = []
number_of_cycles = 4
number_of_sequences = 4

for i in range(number_of_sequences):
    sequence = [[random() < 0.05 for note in range(16)] for beat in range(16)]
    sequences.append(sequence)


After that we simply choose how many times we want to repeat the sequences

In [ ]:
number_of_repeats = 2

melody = []

for k in range(number_of_repeats):
    for sequence in sequences:
        for i in range(number_of_cycles):
            for beat in sequence:
                chord = [0.0 for x in range(note_length_samples)]
                for note_num,should_play_note in enumerate(beat):
                    if should_play_note:
                        chord = [x + y for x, y in zip(chord, notes[note_num])]
                melody.extend(chord)
                melody.extend(rest)
            
Audio(data=melody, rate=sampling_rate)

### Change the timbre

Rather than using simple tones, we can change the timebre by using different waveforms

When we generate our list of notes, lets make them square waves

For good measure we can put the notes through a `tanh` to add a little distortion

In [ ]:
from math import tanh

melody = []

sampling_rate = 44100
beats_per_minute = 120
beat_length = 60 / beats_per_minute
note_length_seconds = beat_length / 8
note_length_samples = int(note_length_seconds * sampling_rate)


notes = []

num_harmonics = 10

for freq in note_freqs:
    note = []
    for i in range(note_length_samples):
        note.append(0.0)
        for k in range(1,num_harmonics,2):
            gain = 1.0 / (freq * (k+1))
            delta = 2.0 * pi * freq * k / fs 
            note[i] += gain*sin(delta * i)
        note[i] = tanh(100.0*note[i])
        
    notes.append(note)
    
rest = [0.0 for x in range(note_length_samples)]

In [ ]:
number_of_repeats = 2

melody = []

for k in range(number_of_repeats):
    for sequence in sequences:
        for i in range(number_of_cycles):
            for beat in sequence:
                chord = [0.0 for x in range(note_length_samples)]
                for note_num,should_play_note in enumerate(beat):
                    if should_play_note:
                        chord = [x + y for x, y in zip(chord, notes[note_num])]
                melody.extend(chord)
                melody.extend(rest)
            
Audio(data=melody, rate=sampling_rate)